#Causal Graph Fuzzy LLM (MIMO)

In [ ]:
!pip install --quiet git+https://github.com/PatriciaLucas/AutoML.git
!pip install --quiet git+https://github.com/petroniocandido/clshq_tk.git
!git clone --quiet https://github.com/FutureLab-DCC/CGF-LLM.git

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.6/309.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 18.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Cloning into 'CGF-LLM'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 186 (delta 38), reused 15 (delta 15), pack-reused 133 (from 1)
Receiving objects: 100% (186/186), 4.79 MiB | 14.83 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch
import sys
sys.path.append('CGF-LLM')
import MCGFLLM, utils

In [ ]:
name_dataset = 'SONDA'
target = 'glo_avg'
df = utils.get_dataset(name = name_dataset)

max_lags = 20
partitions = 30
epochs = 20
database_metrics_path = 'MCGFLLM_metrics.db'
database_size_path = 'MCGFLLM_size.db'

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
name_model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(name_model)

windows = utils.rolling_window(df, 10)

utils.execute("CREATE TABLE IF NOT EXISTS results(model TEXT, name_dataset TEXT, window INT, variable TEXT, predict FLOAT, \
               real FLOAT)", database_metrics_path)
utils.execute("CREATE TABLE IF NOT EXISTS results(model TEXT, name_dataset TEXT, window INT, total_chars FLOAT, train_token_count FLOAT, test_token_count FLOAT, train_char_count FLOAT, \
               test_char_count FLOAT)", database_size_path)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[]

In [ ]:
# Loop through the windows
# Loop through the windows
for i, window in enumerate(windows):
      if i < 0:  # Enter the number of the last window saved in the database here
        print("window already executed")

      else:

        # Exclude constant series.
        for variable in window.columns:
            if window[variable].max() == window[variable].min():
                window = window.drop(variable, axis=1)
                print(f"Variables {variable} were deleted because they are constant.")


        # CAUSAL GRAPH FUZZY
        model_name = 'MCGF-LLM'
        ds, scaler, tokenizer_F, seq, graph = MCGFLLM.fuzzy_causal(window, name_dataset, max_lags, tokenizer, partitions)

        # TEXT
        # model_name = 'MTEXT-LLM'
        # ds, scaler, tokenizer_T, seq = MCGFLLM.text(window, name_dataset, max_lags, tokenizer)

        # CAUSAL GRAPH TEXT
        # model_name = 'MCGTEXT-LLM'
        # ds, scaler, tokenizer, seq, graph = MCGFLLM.causal_text(window, name_dataset, max_lags, tokenizer)

        train_dataset, test_dataset = torch.utils.data.random_split(ds, [0.80, 0.20])

        freeze = True # Freeze GPT-2 weights
        output_size = int(len(window.columns))
        model = MCGFLLM.train_model(train_dataset, name_model, epochs, scaler, output_size, freeze)

        forecasts, real = MCGFLLM.predict(test_dataset, model, tokenizer, target, scaler)

        for row in range(forecasts.shape[0]):
          for v in forecasts.columns:
            utils.execute_insert("INSERT INTO results VALUES(?, ?, ?, ?, ?, ?)", (model_name, name_dataset, i, v, forecasts[v].iloc[row], real[v].iloc[row]), database_metrics_path)

        # Calcula número de tokens e texto
        total_chars, train_token_count, test_token_count, train_char_count, test_char_count = utils.calculate_train_test_sizes(ds, seq, tokenizer)
        utils.execute_insert("INSERT INTO results VALUES(?, ?, ?, ?, ?, ?, ?, ?)", (model_name, name_dataset, i, total_chars, train_token_count, test_token_count, train_char_count, test_char_count), database_size_path)

        print(f'Save window: {i}')

## How to view the results

In [ ]:
avg_metrics = utils.get_metrics_multivariate(database_metrics_path)
avg_metrics

,model,name_dataset,variable,nrmse
0,CGF-LLM,SONDA,diff_avg,0.036764
1,CGF-LLM,SONDA,dir_avg,0.036547
2,CGF-LLM,SONDA,glo_avg,0.200396
3,CGF-LLM,SONDA,humid,0.296802
4,CGF-LLM,SONDA,lux_avg,0.056626
5,CGF-LLM,SONDA,lw_avg,0.145836
6,CGF-LLM,SONDA,par_avg,0.026499
7,CGF-LLM,SONDA,press,0.294254
8,CGF-LLM,SONDA,rain,0.077985
9,CGF-LLM,SONDA,tp_sfc,0.242001


In [ ]:
avg_sizes = utils.get_sizes(database_size_path)
avg_sizes

,model,name_dataset,total_chars,train_token_count,test_token_count,train_char_count,test_char_count
0,CGF-LLM,SONDA,4566914.0,2266650.0,566460.0,3653713.5,913200.5
1,MCGF-LLM,SONDA,3076300.5,1532452.5,383067.5,2461089.5,615211.0
